In [1]:
import torch
from transformers import RobertaTokenizer, RobertaForMultipleChoice, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
import json
import json_lines
import os
from tqdm import tqdm

In [2]:
class MultipleChoiceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_ids, attention_mask, label = self.data[idx]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

In [3]:
answer_map = {'A':0,'B':1,'C':2,'D':3,'E':4}
def load_og_data(file_path):
    data = []
    with open(file_path, 'rb') as f: 
        for item in json_lines.reader(f):
            data.append(item)

    processed_data = []
    for item in data:
        question = item['question']['stem']
        options = [_['text'] for _ in item['question']['choices']]
        examples = []
        for option in options:
            text = question + " " + option
            encoded = tokenizer.encode_plus(
                text,
                truncation=True,
                max_length=512,
                padding='max_length',
                return_attention_mask=True,
                return_tensors='pt'
            )
            examples.append(encoded)
    
        input_ids = torch.stack([example['input_ids'] for example in examples]).squeeze()
        attention_mask = torch.stack([example['attention_mask'] for example in examples]).squeeze()

        label = torch.tensor(answer_map[item['answerKey']])

        processed_data.append((input_ids, attention_mask, label))

    return processed_data

In [4]:
def load_adv_data(file_path):
    with open(file_path, 'r') as f: 
        data = json.load(f)

    processed_data = []
    for item in data:
        question = item['question']
        options = [str(option) for option in item['choice_list']]
        examples = []
        for option in options:
            text = question + " " + option
            encoded = tokenizer.encode_plus(
                text,
                truncation=True,
                max_length=512,
                padding='max_length',
                return_attention_mask=True,
                return_tensors='pt'
            )
            examples.append(encoded)
    
        input_ids = torch.stack([example['input_ids'] for example in examples]).squeeze()
        attention_mask = torch.stack([example['attention_mask'] for example in examples]).squeeze()

        label = torch.tensor(item['label'])

        processed_data.append((input_ids, attention_mask, label))

    return processed_data

In [5]:
model = RobertaForMultipleChoice.from_pretrained('roberta-large')
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# train_data = load_og_data("data/rs_train.jsonl")
# valid_data = load_og_data("data/rs_dev.jsonl")

train_data = load_adv_data("data/adversarial_rs_train.json")
valid_data = load_og_data("data/rs_dev.jsonl")

In [7]:
train_dataset = MultipleChoiceDataset(train_data)
valid_dataset = MultipleChoiceDataset(valid_data)

batch_size = 4

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size)

In [8]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cuda:1"
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 3
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/home/devanshj/miniconda3/envs/anlp/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    average_loss = 0
    for index, batch in tqdm(enumerate(train_loader)):
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)
        outputs = model(**inputs, labels=labels)
        if index ==0:
            print(outputs)
        loss = outputs.loss
        total_loss += loss.item()
        average_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        if index%100==0:
            print("###{}####: Average loss: {}".format(index,average_loss / 100))
            average_loss = 0

    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    preds = []
    true_labels = []
    for batch in tqdm(valid_loader):
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"]
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        preds.extend(torch.argmax(logits, dim=1).detach().cpu().numpy())
        true_labels.extend(labels.numpy())

    acc = accuracy_score(true_labels, preds)
    print(f'Epoch: {epoch+1}, Train loss: {avg_train_loss}, Validation accuracy: {acc}')
model.save_pretrained('/usr1/data/devanshj/brainteaser/checkpoints/roberta_rs_finetuned'.format(epoch))

0it [00:00, ?it/s]

MultipleChoiceModelOutput(loss=tensor(1.6194, device='cuda:1', grad_fn=<NllLossBackward0>), logits=tensor([[-0.1231, -0.2414, -0.1179,  0.0579,  0.0723],
        [ 0.2343, -0.0635,  0.0789, -0.1069, -0.1093],
        [-0.2886,  0.1360,  0.1114, -0.0787,  0.1086],
        [ 0.0443,  0.0033,  0.0247,  0.0093, -0.1758]], device='cuda:1',
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


1it [00:01,  1.67s/it]

###0####: Average loss: 0.016194376945495605


101it [02:17,  1.38s/it]

...

###1600####: Average loss: 0.4259777308237972


1701it [39:06,  1.38s/it]

###1700####: Average loss: 0.5753501329075515


1755it [40:20,  1.38s/it]
100%|██████████| 256/256 [02:02<00:00,  2.08it/s]


Epoch: 3, Train loss: 0.5555712995296058, Validation accuracy: 0.5494613124387855
